# Introduction to Bash Scripting

Author: Ken Youens-Clark

In this notebook, we are going to get started with some basic Bash scripting. Bash scripting is useful for bioinformatics because all scripts we submit to run jobs on the HPC are written in Bash. 

Here's an overview:
- Learn how to write a basic script
- Get data into your programs
- Learn about For and While loops

-----


### Getting Started

Before we get started you will need to set your netid and then go into the directory for this assignment under bh_class.

You will need to rerun this section each time you come back to this notebook to reset the variables.

Remember, our notebooks work in the current working directory -- and when you login to the HPC this automatically is your home directory. You will need to move to the project directory. The next two cells set your netid and the project directory (be sure to replace "MY_NETID" with your actual netid). Then we will change into that directory for our exercise.

In [ ]:
# Change "MY_NETID" to your netid below, and run this cell
netid = "MY_NETID"

In [ ]:
# Set the working directory and change into this directory
work_dir = "/xdisk/bhurwitz/bh_class/" + netid + "/assignments/03_bash_scripting"
%cd $work_dir

### Section 1: Hello World!

#### Shebang

Scripting languages \(sh, bash, Perl, Python, Ruby, etc.\) are generally distinguished by the fact that the "program" is a regular file containing plain text that is interpreted into machine code at the time you run it.  Basically a "script" is a plain text file that is often executable by virtue of having the executable bit\(s\) turned on \(cf. "Permissions"\).  It does not have to be executable, however.  It's acceptable to put some commands in a file and simply tell the appropriate program to interpret the file:

In [ ]:
!echo "echo Hello, World" > hello.sh
!sh hello.sh

#### What do you get?

You should get this:
```
Hello, World
```

But it looks cooler to have it run as a script by making it executable.

In [ ]:
# change the script so it is executable.
!chmod +x hello.sh
# run the script
!./hello.sh

#### What is a shebang line?

The interesting thing about shell scripts is that you can often times run commands without telling the computer what you want to use to run the code. The reason is that the parent shell guesses that the script is written for the same shell. You can get into trouble is the code is written in another language, like python, or for a different shell. 

Therefore, it is always best practice to add a shebang line that will indicate the absolute path to a program like "/bin/bash" or "/usr/local/bin/python3". 

In [ ]:
# To find out where bash is installed you can type:
!which bash

#### Let's Make A Script!

Let's make our script say "Hello" to some people. Let's write a script with the following lines of code, and output. Note that I included line numbers here for your reference, but the code you write should not have them, see the example below:

```
$ cat -n hello2.sh
     1    #!/bin/bash
     2
     3    NAME="Newman"
     4    echo "Hello," $NAME
     5    NAME="Jerry"
     6    echo "Hello, $NAME"

$ ./hello2.sh
Hello, Newman
Hello, Jerry
```

We can use python to print the code above into a file for us (see below). The three single quotes indicate the beginning and end of a code block, that we are putting into to a variable called code_for_script. We then open a file called hello2.sh and print that variable to the file. 


In [ ]:
my_code = '''#!/bin/bash

NAME="Newman"
echo "Hello," $NAME
NAME="Jerry"
echo "Hello, $NAME"
'''

with open('hello2.sh', mode='w') as file:
    file.write(my_code)

In [ ]:
'''
Type the command below, and run the cell
1.1 write a command to make hello2.sh executable
1.2 write a command to run the hello2.sh script
'''

#### What is this script doing?

I've created a variable called `NAME` to hold the string "Newman" and print it.  Notice there is no `$` when assigning to the variable, only when you use it. The value of `NAME` can be changed at any time.  You can print it out like on line 4 as it's own argument to `echo` or inside of a string like on line 6.  Notice that the version on line 4 puts a space between the arguments to `echo`.

Because all the variables from the environment \(see `env`\) are uppercase \(e.g., `$HOME` and `$USER`\), I tend to use all-caps myself, but this did lead to a problem once when I named a variable `PATH` and then overwrote the actual `PATH` and then my program stopped working entirely as it could no longer find any of the programs it needed.  Just remember that everything in Unix is case-sensitive, so `$Name` is an entirely different variable from `$name`.

When assigning a variable, you can have NO SPACES around the `=` sign:

```
$ NAME1="Doge"
$ echo "Such $NAME1"
Such Doge
$ NAME2 = "Doge"
-bash: NAME2: command not found
$ echo "Such $NAME2"
Such
```

### Section 2: Getting Data Into Your Program: Arguments

We would like to get the NAME from the user rather than having it hardcoded in the script.  I'll show you three ways our script can take in data from outside:

1. Command-line arguments, both positional \(i.e., the first one, the second one, etc.\) or named \(e.g., `-n NAME`\)
2. The environment
3. Reading a configuration file

First we'll cover the command-line arguments which are available through a few variables:

* `$#`: The number \(think "\#" == number\) of arguments
* `$@`: All the arguments in a single string
* `$0`: The name of the script
* `$1, $2`: The first argument, the second argument, etc.

A la:

```
$ cat -n args.sh
     1    #!/bin/bash
     2
     3    echo "Num of args    : \"$#\""
     4    echo "String of args : \"$@\""
     5    echo "Name of program: \"$0\""
     6    echo "First arg      : \"$1\""
     7    echo "Second arg     : \"$2\""
```

In [ ]:
# Now you try. Create the script above by inputing the code into the code block below
# 2.1
code_for_script = '''#!/bin/bash
  

'''

with open('args.sh', mode='w') as file:
    file.write(code_for_script)

In [ ]:
'''
Type the command below, and run the cell
2.2 write a command to make args.sh executable
2.3 write a command to run the args.sh script
'''

In [ ]:
# Let's try running the script with no input
!./args.sh

In [ ]:
# Let's try running the script with the input "foo"
!./args.sh foo

In [ ]:
# Let's try running the script with the input "foo bar"
!./args.sh foo bar

#### What did you get?

You should have seen something like this:

```
$ ./args.sh
Num of args    : "0"
String of args : ""
Name of program: "./args.sh"
First arg      : ""
Second arg     : ""

$ ./args.sh foo
Num of args    : "1"
String of args : "foo"
Name of program: "./args.sh"
First arg      : "foo"
Second arg     : ""

$ ./args.sh foo bar
Num of args    : "2"
String of args : "foo bar"
Name of program: "./args.sh"
First arg      : "foo"
Second arg     : "bar"
```

### Section 3: Iterating through arguments on the command line

If you would like to iterate over all the arguments, you can use `$@` like so:

```
$ cat -n args2.sh
     1    #!/bin/bash
     2
     3    if [[ $# -lt 1 ]]; then
     4        echo "There are no arguments"
     5    else
     6        i=0
     7        for ARG in "$@"; do
     8            let i++
     9            echo "$i: $ARG"
    10        done
    11    fi
```

Here I'm using a conditional at line 3 to check if the script has any arguments. If the number of arguments \(`$#`\) is less than \(`-lt`\) 1, then let the user know there is nothing to show; otherwise \(`else`\) do the next block of code.  

The `for` loop on line 7 works by splitting the argument string \(`$@`\) on spaces just like the command line does.  Both `for` and `while` loops require the `do/done` pair to delineate the block of code \(some languages use `{}`).  Along those lines, line 11 is the close of the `if` -- "if" spell backwards.

The other bit of magic I threw in was a counter variable \(which I always use lowercase `i` \["integer"\], `j` if I needed an inner-counter and so on\) which is initialized to "0" on line 6.  I increment it, I could have written `$i=$(($i + 1))`, but it's easier to use the `let i++` shorthand.  

In [ ]:
# Now you try. Create the script above by inputting the code into the code block below
# 2.4
code_for_script = '''#!/bin/bash
  

'''

with open('args2.sh', mode='w') as file:
    file.write(code_for_script)

In [ ]:
# Now run the args2.sh script to test it.
!chmod +x args2.sh
!./args2.sh
!./args2.sh foo
!./args2.sh foo bar "baz quux"

#### What do you see?

Notice that "baz quux" seen as a single argument because it was placed in quotes; otherwise arguments are separated by spaces.

#### Sidebar: Make It Pretty \(or else\)

Note that indentation doesn't matter as the program below works, but, honestly, which one is easier for you to read?

```
$ cat -n args3.sh
     1    #!/bin/bash
     2
     3    if [[ $# -lt 1 ]]; then
     4    echo "There are no arguments"
     5    else
     6    i=0
     7    for ARG in "$@"; do
     8    let i++
     9    echo "$i: $ARG"
    10    done
    11    fi
$ ./args3.sh foo bar
1: foo
2: bar
```

#### Sidebar: Catching Common Errors \(set -u\)

bash is a notoriously easy language to write incorrectly.  One step you can take to ensure you don't misspell variables is to add `set -u` at the top of your script.  E.g., if you type `echo $HOEM` on the command line, you'll get no output or warning that you misspelled the `$HOME` variable unless you `set -u`:

```
$ echo $HOEM

$ set -u
$ echo $HOEM
-bash: HOEM: unbound variable
```

#### You can avoid bugs with set -u, but there are other gotchas...

set -u tells bash to complain when you use a variable that was never initialized to some value. This is like putting on your helmet.  It's not a requirement \(depending on which state you live in\), but you absolutely should do this because there might come a day when you misspell a variable.  Note that this will not save you from as error like this:

```
$ cat -n set-u-bug1.sh
     1    #!/bin/bash
     2
     3    set -u
     4
     5    if [[ $# -gt 0 ]]; then
     6      echo $THIS_IS_A_BUG; # never initialized
     7    fi
     8
     9    echo "OK";

$ ./set-u-bug1.sh
OK

$ ./set-u-bug1.sh foo
./set-u-bug1.sh: line 6: THIS_IS_A_BUG: unbound variable
```

You can see that the first execution of the script ran just fine. There is a bug on line 6, but bash didn't catch it because that line did not execute.  On the second run, the error occurred, and the script blew up.  \(FWIW, this is a problem in Python, too.\)


#### Here's another pernicious error:

```
$ cat -n set-u-bug2.sh
     1    #!/bin/bash
     2
     3    set -u
     4
     5    GREETING="Hi"
     6    if [[ $# -gt 0 ]]; then
     7      GRETING=$1 # misspelled
     8    fi
     9
    10    echo $GREETING
$ ./set-u-bug2.sh
Hi
$ ./set-u-bug2.sh Hello
Hi
```

Did `set -u` help us? Tell me why or why not by creating a README1.txt file about this.

NB: I highly recommend you use the program `shellcheck` \([https://www.shellcheck.net/\](https://www.shellcheck.net/%29\) to find errors in your bash code.

In [ ]:
# Write a README1.txt file about the error above, did set -u help? why or why not?
# 2.5
!echo "Describe the error here" > README1.txt

### Section 3: Our First Argument

AT LAST, let's return to our "hello" script!

```
$ cat -n hello3.sh
     1    #!/bin/bash
     2
     3    echo "Hello, $1!"
$ ./hello3.sh Captain
Hello, Captain!
```

This should make perfect sense now.  We are simply saying "hello" to the first argument, but what happens if we provide no arguments?

In [ ]:
# Now you try. Create the script above by inputting the code into the code block below
# 3.1
code_for_script = '''#!/bin/bash
  

'''

with open('hello3.sh', mode='w') as file:
    file.write(code_for_script)

In [ ]:
'''
Type the command below, and run the cell
3.2 write a command to make hello3.sh executable
3.3 run hello.sh with no arguments
'''


#### Checking the Number of Arguments

Well, that looks bad.  We should check that the script has the proper number of arguments which is 1:

```
$ cat -n hello4.sh
     1    #!/bin/bash
     2
     3    if [[ $# -ne 1 ]]; then
     4        printf "Usage: %s NAME\n" "$(basename "$0")"
     5        exit 1
     6    fi
     7
     8    echo "Hello, $1!"

$ ./hello4.sh
Usage: hello4.sh NAME

$ ./hello4.sh Captain
Hello, Captain!

$ ./hello4.sh Captain Picard
Usage: hello4.sh NAME
```

In [ ]:
# Now you try. Create the script above by inputting the code into the code block below
# 3.4
code_for_script = '''#!/bin/bash
  

'''

with open('hello4.sh', mode='w') as file:
    file.write(code_for_script)

In [ ]:
'''
Type the commands below, and run the cell
3.5 write a command to make hello4.sh executable
3.6 run hello4.sh with the example arguments above
'''

In [ ]:
# Now, tell me line by line what the hello4.sh script is doing. Create a README2.txt file with your explaination.
!echo "This is my explaination." > README2.txt

#### What else can we do?

Here is an alternate way to write this script:

```
$ cat -n hello5.sh
     1    #!/bin/bash
     2
     3    if [[ $# -eq 1 ]]; then
     4        NAME=$1
     5        echo "Hello, $NAME!"
     6    else
     7        printf "Usage: %s NAME\n" "$(basename "$0")"
     8        exit 1
     9    fi
```

Here I check on line 3 if there is just one argument, and the `else` is devoted to handling the error; however, I prefer to check for all possible errors at the beginning and `exit` the program quickly. This also has the effect of keeping my code as far left on the page as possible.

In [ ]:
# Now you try. Create the script above by inputting the code into the code block below
# 3.7
code_for_script = '''#!/bin/bash
  

'''

with open('hello5.sh', mode='w') as file:
    file.write(code_for_script)

In [ ]:
'''
Type the commands below, and run the cell
3.7 write a command to make hello5.sh executable
3.8 run hello5.sh with the example arguments above
'''

#### Sidebar: Saving Function Results

In the previous script, you may have noticed `$(basename "$0")`.  I was passing the script name \(`$0`\) to the function `basename` and then passing that to the `printf` function.  

To call a function in bash and save the results into a variable or use the results as an argument, we can use either backticks \(\`\`\) \(under the `~` on a US keyboard\) or `$()`.  I find backticks to be too similar to single quotes, so I prefer the latter.  To demonstrate:

    $ ls | head
    args.sh*
    args2.sh*
    args3.sh*
    basic.sh*
    hello.sh*
    hello2.sh*
    hello3.sh*
    hello4.sh*
    hello5.sh*

    $ FILES=`ls | head`
    $ echo $FILES
    
    args.sh args2.sh args3.sh basic.sh hello.sh hello2.sh hello3.sh hello4.sh hello5.sh


### Section 4: Providing Default Argument Values

Here is how you can provide a default value for an argument with `:-`:

```
$ cat -n hello6.sh
     1    #!/bin/bash
     2
     3    echo "Hello, ${1:-Stranger}!"

$ ./hello6.sh
Hello, Stranger!

$ ./hello6.sh Govnuh
Hello, Govnuh!
```

In [ ]:
# Now you try. Create the script above by inputting the code into the code block below
# 4.1
code_for_script = '''#!/bin/bash
  

'''

with open('hello6.sh', mode='w') as file:
    file.write(code_for_script)

In [ ]:
'''
Type the commands below, and run the cell
4.2 write a command to make hello6.sh executable
4.3 run hello6.sh with the example arguments above
'''

### Section 5: Arguments from the Environment

You can also use look in the environment for argument values.  For instance, we could accept the `NAME` as either the first argument to the script \(`$1`\) or the `$USER` from the environment:

```
$ cat -n hello7.sh
     1    #!/bin/bash
     2
     3    NAME=${1:-$USER}
     4    [[ -z "$NAME" ]] && NAME='Stranger'
     5    echo "Hello, $NAME
$ ./hello7.sh
Hello, bhurwitz
$ ./hello7.sh Barbara
Hello, Barbara
```

In [ ]:
# Now you try. Create the script above by inputting the code into the code block below
# 5.1
code_for_script = '''#!/bin/bash
  

'''

with open('hello7.sh', mode='w') as file:
    file.write(code_for_script)

In [ ]:
'''
Type the commands below, and run the cell
5.2 write a command to make hello7.sh executable
5.3 run hello7.sh with the example arguments above (or try your own!)
'''

#### Let's play around with this...

What happens when you override a variable like so?

```
$ USER=Bart ./hello7.sh
Hello, Bart
$ ./hello7.sh
Hello, bhurwitz
```

In [ ]:
# Write a README file that describes what is happening above.
# 5.4
!echo "This is my explaination." > README3.txt

### Section 6: Exporting Values to the Environment

Notice that I can set `USER` for the first run to "Bart," but the value returns to "bhurwitz" on the next run.  I can permanently set a value in the environment by using the `export` command.  Here is a version of the script that looks for an environmental variable called `WHOM` \(please do not override your `$USER` name in the environment as things will break\):

```
$ cat -n hello8.sh
     1    #!/bin/bash
     2
     3    echo "Hello, ${WHOM:-Marie}"
$ ./hello8.sh
Hello, Marie
```

As before I can set it temporarily:

```
$ WHOM=Doris ./hello8.sh
Hello, Doris
$ ./hello8.sh
Hello, Marie
```

Now I will `export WHOM` so that it persists:

```
$ WHOM=Doris
$ export WHOM
$ ./hello8.sh
Hello, Doris
$ ./hello8.sh
Hello, Doris
```

To remove `WHOM` from the environment, use `unset`:

```
$ unset WHOM
$ ./hello8.sh
Hello, Marie
```

#### Let's try working with environmental variables

Some programs rely heavily on environmental variables \(e.g., bioinformatics programs like Centrifuge\) for arguments.  Here is a short script to illustrate how you would use such a program:

```
$ cat -n hello9.sh
     1    #!/bin/bash
     2
     3    WHOM="Who's on first" ./hello8.sh
     4    WHOM="What's on second"
     5    export WHOM
     6    ./hello8.sh
     7    WHOM="I don't know's on third" ./hello8.sh

$ ./hello9.sh
Hello, Who's on first
Hello, What's on second
Hello, I don't know's on third
```

#### Required and Optional Arguments

Now we're going to accept two arguments, "GREETING" and "NAME" while providing defaults for both:

```
$ cat -n positional.sh
     1    #!/bin/bash
     2
     3    set -u
     4
     5    GREETING=${1:-Hello}
     6    NAME=${2:-Stranger}
     7
     8    echo "$GREETING, $NAME"

```

In [ ]:
# Now you try. Create the script above by inputting the code into the code block below
# 6.1 
code_for_script = '''#!/bin/bash
  

'''

with open('positional.sh', mode='w') as file:
    file.write(code_for_script)

In [ ]:
# Test the script with the following arguments
!chmod +x positional.sh
!./positional.sh
!./positional.sh Howdy
!./positional.sh Howdy Padnuh
!./positional.sh "" Pahnuh

#### You should get something like this. Did it work?

```
$ ./positional.sh
Hello, Stranger
$ ./positional.sh Howdy
Howdy, Stranger
$ ./positional.sh Howdy Padnuh
Howdy, Padnuh
$ ./positional.sh "" Pahnuh
Hello, Pahnuh
```

You'll notice that if I want to use the default argument for the greeting, I have to pass an empty string `""`.

#### What if I want to require at least one argument?

```
$ cat -n positional2.sh
     1    #!/bin/bash
     2
     3    set -u
     4
     5    if [[ $# -lt 1 ]]; then
     6        printf "Usage: %s GREETING [NAME]\n" "$(basename "$0")"
     7        exit 1
     8    fi
     9
    10    GREETING=$1
    11    NAME=${2:-Stranger}
    12
    13    echo "$GREETING, $NAME"
```


In [ ]:
# Now you try. Create the script above by inputting the code into the code block below
# 6.2
code_for_script = '''#!/bin/bash
set -u
    
'''

with open('positional2.sh', mode='w') as file:
    file.write(code_for_script)

In [ ]:
# Try it out
!chmod +x positional2.sh
!./positional2.sh "Good Day"
!./positional2.sh "Good Day" "Kind Sir"

#### Did you get something like this?

```
$ ./positional2.sh "Good Day"
Good Day, Stranger
$ ./positional2.sh "Good Day" "Kind Sir"
Good Day, Kind Sir
```

It's also important to note the subtle hints given to the user in the "Usage" statement.  `[NAME]` has square brackets to indicate that it is an option, but `GREETING` does not to say it is required.  As noted before I wanted to use the GREETING "Good Day," so I had to put it in quotes so that the shell would not interpret them as two arguments.  Same with the NAME "Kind Sir."

In [ ]:
!./positional2.sh Good Day Kind Sir

In [ ]:
# Write a README file below that explains the output for the command above.
# 6.3
!echo "This is my explaination." > README4.txt

#### Not Too Few, Not Too Many \(Goldilocks\)

Hmm, maybe we should detect that the script had too many arguments?

```
$ cat -n positional3.sh
     1    #!/bin/bash
     2
     3    set -u
     4
     5    if [[ $# -lt 1 ]] || [[ $# -gt 2 ]]; then
     6        printf "Usage: %s GREETING [NAME]\n" "$(basename "$0")"
     7        exit 1
     8    fi
     9
    10    GREETING=$1
    11    NAME=${2:-Stranger}
    12
    13    printf "%s, %s\n" "$GREETING" "$NAME"
```

In [ ]:
# Now you try. Create the script above by inputting the code into the code block below
# 6.4
code_for_script = '''#!/bin/bash
  

'''

with open('positional3.sh', mode='w') as file:
    file.write(code_for_script)

In [ ]:
# Test with the following arguments
!chmod +x positional3.sh
!./positional3.sh Good Day Kind Sir
!./positional3.sh "Good Day" "Kind Sir"

#### Did you see something like this:

```
$ ./positional3.sh Good Day Kind Sir
Usage: positional3.sh GREETING [NAME]
$ ./positional3.sh "Good Day" "Kind Sir"
Good Day, Kind Sir
```

To check for too many arguments, I added an "OR" \(the double pipes `||`\) and another conditional \("AND" is `&&`\).  I also changed line 13 to use a `printf` command to highlight the importance of quoting the arguments _inside the script_ so that bash won't get confused.  Try it without those quotes and try to figure out why it's doing what it's doing.  I highly recommend using the program "shellcheck" \([https://github.com/koalaman/shellcheck](https://github.com/koalaman/shellcheck)\) to find mistakes like this.


### Section 7: Named Arguments To The Rescue

I hope maybe by this point you're thinking that the script is getting awfully complicated just to allow for a combination of required an optional arguments all given in a particular order.  You can manage with 1-3 positional arguments, but, after that, we really need to have named arguments and/or flags to indicate how we want to run the program.  A named argument might be `-f mouse.fa` to indicate the value for the `-f` \("file," probably\) argument is "mouse.fa," whereas a flag like `-v` might be a yes/no \("Boolean," if you like\) indicator that we do or do not want "verbose" mode.  You've encountered these with programs like `ls -l` to indicate you want the "long" directory listing or `ps -u $USER` to indicate the value for `-u` is the `$USER`.

The best thing about named arguments is that they can be provided in any order:

```
$ ./named.sh -n Patch -g "Good Boy"
Good Boy, Patch!
```

Some may have values, some may be flags, and you can easily provide good defaults to make it easy for the user to provide the bare minimum information to run your program. Here is a version that has named arguments:

```
$ cat -n named.sh
     1	#!/bin/bash
     2	
     3	set -u
     4	
     5	GREETING=""
     6	NAME="Stranger"
     7	EXCITED=0
     8	
     9	function USAGE() {
    10	    printf "Usage:\n  %s -g GREETING [-e] [-n NAME]\n\n" $(basename $0)
    11	    echo "Required arguments:"
    12	    echo " -g GREETING"
    13	    echo
    14	    echo "Options:"
    15	    echo " -n NAME ($NAME)"
    16	    echo " -e Print exclamation mark (default yes)"
    17	    echo 
    18	    exit ${1:-0}
    19	}
    20	
    21	[[ $# -eq 0 ]] && USAGE 1
    22	
    23	while getopts :g:n:eh OPT; do
    24	  case $OPT in
    25	    h)
    26	      USAGE
    27	      ;;
    28	    e)
    29	      EXCITED=1
    30	      ;;
    31	    g)
    32	      GREETING="$OPTARG"
    33	      ;;
    34	    n)
    35	      NAME="$OPTARG"
    36	      ;;
    37	    :)
    38	      echo "Error: Option -$OPTARG requires an argument."
    39	      exit 1
    40	      ;;
    41	    \?)
    42	      echo "Error: Invalid option: -${OPTARG:-""}"
    43	      exit 1
    44	  esac
    45	done
    46	
    47	[[ -z "$GREETING" ]] && USAGE 1
    48	PUNCTUATION="."
    49	[[ $EXCITED -ne 0 ]] && PUNCTUATION="!"
    50	
    51	echo "$GREETING, $NAME$PUNCTUATION"

```

In [ ]:
# Now you try. Create the script above by inputting the code into the code block below
# 7.1
code_for_script = '''#!/bin/bash
  

'''

with open('named.sh', mode='w') as file:
    file.write(code_for_script)

In [ ]:
# try to run it without arguments:
!chmod +x named.sh
!./named.sh

In [ ]:
# Test the code using the examples above

#### When run without arguments or with the `-h` flag, it produces a help message. 

```
$ ./named.sh
Usage:
  named.sh -g GREETING [-e] [-n NAME]

Required arguments:
 -g GREETING

Options:
 -n NAME (Stranger)
 -e Print exclamation mark (default yes)
```

Our script just got much longer but also more flexible.  I've written a hundred shell scripts with just this as the template, so you can, too.  Go search for how `getopt` works and copy-paste this for your bash scripts, but the important thing to understand about `getopt` is that flags that take arguments have a `:` after them \(`g:` == "-g something"\) and ones that do not, well, do not \(`h` == "-h" == "please show me the help page\).  Both the "h" and "e" arguments are flags:

```
$ ./named.sh -n Patch -g "Good Boy"
Good Boy, Patch.
$ ./named.sh -n Patch -g "Good Boy" -e
Good Boy, Patch!
```

I've introduced a new function called `USAGE` that prints out the "Usage" statement so that it can be called when:

* the script is run with no arguments \(line 21\)
* the script is run with the "-h" flag \(lines 25-26\)
* the script is run with bad input \(line 47\)

I initialized the NAME to "Stranger" \(line 6\) and then let the user know in the "Usage" what the default value will be.  When checking the GREETING in line 44, I'm actually checking that the length of the value is greater than zero because it's possible to run the script like this:

```
$ ./named01.sh -g ""
```

Which would technically pass muster but does not actually meet our requirements.

### Section 8:  Reading a Configuration File

The last way I'll show you to get data into your program is to read a configuration file.  This builds on the earlier example of using `export` to put values into the environment:

```
$ cat -n config1.sh
     1	export NAME="Merry Boy"
     2	export GREETING="Good morning"
$ cat -n read-config.sh
     1	#!/bin/bash
     2
     3	source config1.sh
     4	echo "$GREETING, $NAME!"
$ ./read-config.sh
Good morning, Merry Boy!
```

In [ ]:
# Now you try. Create the script above by inputting the code into the code block below
# 8.1
code_for_script = '''
  

'''

with open('config1.sh', mode='w') as file:
    file.write(code_for_script)

In [ ]:
# now write the read-config.sh file above that reads the config file 
# 8.2
code_for_script = '''#!/bin/bash
  

'''

with open('read-config.sh', mode='w') as file:
    file.write(code_for_script)


In [ ]:
!chmod +x config1.sh
!chmod +x read-config.sh

In [ ]:
# Test the code using the examples above

#### To make this more flexible, let's pass the config file as an argument:

```
$ cat -n read-config2.sh
     1	#!/bin/bash
     2
     3	CONFIG=${1:-config1.sh}
     4	if [[ ! -f "$CONFIG" ]]; then
     5	    echo "Bad config \"$CONFIG\""
     6	    exit 1
     7	fi
     8
     9	source $CONFIG
    10	echo "$GREETING, $NAME!"
```

In [ ]:
# now write the read-config2.sh file above that reads the config file
# 8.3 
code_for_script = '''#!/bin/bash
  

'''

with open('read-config2.sh', mode='w') as file:
    file.write(code_for_script)


In [ ]:
# create a config2.sh file with the following:
# export NAME="Francis"
# export GREETING="Salut"
# 8.4
code_for_script = '''
  

'''

with open('config2.sh', mode='w') as file:
    file.write(code_for_script)

In [ ]:
# Try it out with the following tests
!chmod +x read-config2.sh
!chmod +x config2.sh
!./read-config2.sh
!./read-config2.sh config2.sh
!./read-config2.sh foo

#### You should see something like the following:

```
$ ./read-config2.sh
Good morning, Merry Boy!
$ cat -n config2.sh
     1	export NAME="François"
     2	export GREETING="Salut"
$ ./read-config2.sh config2.sh
Salut, François!
$ ./read-config2.sh foo
Bad config "foo"
```

### Section 9: For Loops

Often we want to do some set of actions for all the files in a directory or all the identifiers in a file.  You can use a `for` loop to iterate over the values in some command that returns a list of results:

```
$ for FILE in *.sh; do echo "FILE = $FILE"; done
FILE = args.sh
FILE = args2.sh
FILE = args3.sh
FILE = basic.sh
FILE = hello.sh
FILE = hello2.sh
FILE = hello3.sh
FILE = hello4.sh
FILE = hello5.sh
FILE = hello6.sh
FILE = named.sh
FILE = positional.sh
FILE = positional2.sh
FILE = positional3.sh
FILE = set-u-bug1.sh
FILE = set-u-bug2.sh
```

Here it is in a script:

```
$ cat -n for.sh
     1    #!/bin/bash
     2
     3    set -u
     4
     5    DIR=${1:-$PWD}
     6
     7    if [[ ! -d "$DIR" ]]; then
     8        echo "$DIR is not a directory"
     9        exit 1
    10    fi
    11
    12    i=0
    13    for FILE in $DIR/*; do
    14        let i++
    15        printf "%3d: %s\n" $i "$FILE"
    16    done
```

On line 5, I default `DIR` to the current working directory which I can find with the environmental variable `$PWD` \(print working directory\).  I check on line 7 that the argument is actually a directory with the `-d` test \(`man test`\).  The rest should look familiar.

In [ ]:
# now write the for.sh script.
# 9.1
code_for_script = '''#!/bin/bash
  

'''

with open('for.sh', mode='w') as file:
    file.write(code_for_script)

In [ ]:
!chmod +x for.sh
!./for.sh | head

### Section 10: While Loops

The proper way to read a file line-by-line is with `while`:

```
$ cat -n while.sh
     1    #!/bin/bash
     2
     3    FILE=${1:-'srr.txt'}
     4    while read -r LINE; do
     5        echo "LINE \"$LINE\""
     6    done < "$FILE"
```

In [ ]:
# now write the while.sh script.
# 10.1
code_for_script = '''#!/bin/bash
  

'''

with open('while.sh', mode='w') as file:
    file.write(code_for_script)

In [ ]:
# create a file with a few SRR ids (identifiers from the sequence read archive)
!echo -e "SRR311596\nSRR516222\nSRR919365" > srr.txt
!chmod +x while.sh
!./while.sh srr.txt

In [ ]:
# now create a metadata.txt file
!echo "GD.Spr.C.8m.fa -17.92522,146.14295" >> metadata.txt
!echo "GF.Spr.C.9m.fa -16.9207,145.9965833" >> metadata.txt
!./while.sh metadata.txt

#### Another advantage is that `while` can break the line into fields:

```
$ cat -n while2.sh
     1    #!/bin/bash
     2
     3    FILE='metadata.txt'
     4    while read -r SITE LOC; do
     5        echo "$SITE is located at \"$LOC\""
     6    done < "$FILE"
```

In [ ]:
# now write the while2.sh script.
# 10.2
code_for_script = '''#!/bin/bash


'''

with open('while2.sh', mode='w') as file:
    file.write(code_for_script)

In [ ]:
# Try it out
!chmod +x while2.sh
!./while2.sh
!./while2.sh metadata.txt

#### Do you see something like this?

```
$ ./while2.sh metadata.txt
GD.Spr.C.8m.fa is located at "-17.92522,146.14295"
GF.Spr.C.9m.fa is located at "-16.9207,145.9965833"
```


### The End!

Last step, copy your completed Jupyter notebook into your assignments directory. Be sure to save your notebook first.

In [ ]:
!cp ~/hw03_bash_scripting.ipynb /xdisk/bhurwitz/bh_class/$netid/assignments/hw03_bash_scripting.ipynb